In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install contractions

     |████████████████████████████████| 317kB 18.2MB/s 
     |████████████████████████████████| 245kB 47.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81701 sha256=46093418470c1acde0973e307279157b46aec2574bc52c2be0b090646ddcf875
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
import numpy as np

import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from keras.utils import np_utils
import contractions
import numpy as np
import re
import tqdm
import unicodedata
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.3.0
Eager mode:  True
Hub version:  0.10.0
GPU is available


#### Load preprocessed data 

In [ ]:
import pickle
##load pre-processed data
with open('/content/gdrive/My Drive/upwork/train_content_new.pkl','rb') as f:
  train_content = pickle.load(f)
with open('/content/gdrive/My Drive/upwork/train_rating_new.pkl','rb') as f:
  train_rating = pickle.load(f)
with open('/content/gdrive/My Drive/upwork/test_content_new.pkl','rb') as f:
  test_content = pickle.load(f)
with open('/content/gdrive/My Drive/upwork/test_rating_new.pkl','rb') as f:
  test_rating = pickle.load(f)

Basic Text Pre-processing
We do minimal text pre-processing here since we are using deep learning models and not count-based methods. Steps include the following:



Converting accented characters

*   Fixing contractions
*   Removing special characters
*   Converting accented characters

Note : For some models we don't use any pre-processing like BERT!

In [ ]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def pre_process_corpus(docs):
    norm_docs = []
    for doc in tqdm.tqdm(docs):
        doc = doc.translate(doc.maketrans("\n\t\r", "   "))
        doc = doc.lower()
        doc = remove_accented_chars(doc)
        doc = contractions.fix(doc)
        # lower case and remove special characters\whitespaces
        doc = re.sub(r'[^a-zA-Z0-9\s]', ' ', doc, re.I|re.A)
        doc = re.sub(' +', ' ', doc)
        doc = doc.strip()  
        norm_docs.append(doc)
    return norm_docs

In [ ]:
%%time
train_content = pre_process_corpus(train_content)
test_content = pre_process_corpus(test_content)

100%|██████████| 50919/50919 [00:06<00:00, 7513.96it/s]

CPU times: user 1min 6s, sys: 861 ms, total: 1min 7s
Wall time: 1min 7s


### convert target variable to one hot encoding since its a multiclass classification problem

In [ ]:
train_rating = np_utils.to_categorical(train_rating)
test_rating = np_utils.to_categorical(test_rating)

### Building input pipeline

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_content, train_rating))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

### Build a NNLM Embedding Layer

In [ ]:
model = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(model, output_shape=[128], input_shape=[], 
                           dtype=tf.string, trainable=True)

### Build Model Architecture

In [ ]:

model = tf.keras.models.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.15))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.15))
model.add(tf.keras.layers.Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               124642688 
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 774       
Total params: 124,676,486
Trainable params: 124,676,486
Non-trainable params: 0
__________________________________________

### load prerained weights if available

In [ ]:
epochs = 100
filepath="/content/gdrive/My Drive/upwork/weights/model.h5"
model.load_weights(filepath)

### Define callbacks 

In [ ]:

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                      patience=3,
                                      restore_best_weights=True,
                                      verbose=1)
    


### Train the model

In [ ]:
for epoch in range(epochs):
  model.fit(train_dataset,
          validation_steps = 0.05,
          epochs=5, 
          shuffle=True,
          callbacks=[es,model_checkpoint_callback],
          verbose=1)
  model.save(filepath)

Epoch 1/5
7161/7161 [==============================] - ETA: 0s - loss: 0.0512 - accuracy: 0.9833WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


7161/7161 [==============================] - 535s 75ms/step - loss: 0.0512 - accuracy: 0.9833
Epoch 2/5
7161/7161 [==============================] - ETA: 0s - loss: 0.0515 - accuracy: 0.9832WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


7161/7161 [==============================] - 536s 75ms/step - loss: 0.0515 - accuracy: 0.9832
Epoch 3/5
7161/7161 [==============================] - ETA: 0s - loss: 0.0488 - accuracy: 0.9842WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


7161/7161 [==============================] - 540s 75ms/step - loss: 0.0488 - accuracy: 0.9842
Epoch 4/5
7161/7161 [==============================] - ETA: 0s - loss: 0.0494 - accuracy: 0.9838WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


7161/7161 [==============================] - 537s 75ms/step - loss: 0.0494 - accuracy: 0.9838
Epoch 5/5
7161/7161 [==============================] - ETA: 0s - loss: 0.0496 - accuracy: 0.9839WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


7161/7161 [==============================] - 537s 75ms/step - loss: 0.0496 - accuracy: 0.9839
Epoch 1/5
7161/7161 [==============================] - ETA: 0s - loss: 0.0496 - accuracy: 0.9840WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


7161/7161 [==============================] - 533s 74ms/step - loss: 0.0496 - accuracy: 0.9840
Epoch 2/5
1396/7161 [====>.........................] - ETA: 7:08 - loss: 0.0473 - accuracy: 0.9843

In [ ]:
model.save('/content/gdrive/My Drive/upwork/weights/new_model.h5')